# Example: Single Agent() 

Install SUMO, SMARTS, required packages, and build the scenarios.

In [ ]:
# Setup SUMO=1.10.0
!apt-get update --fix-missing
!apt-get install -y libspatialindex-dev xorg
%pip install --upgrade pip wheel
%pip install eclipse-sumo==1.10.0
%env SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo

# Install SMARTS
!rm -rf /content/SMARTS/
!git clone https://github.com/huawei-noah/SMARTS
%cd /content/SMARTS 
!git checkout ipynb-1
%pip install -e .[camera-obs]
import sys
sys.path.insert(0, '/content/SMARTS/')

# Build scenarios
!scl scenario build-all --clean /content/SMARTS/scenarios/loop

Define the agent and the environment loop.

In [ ]:
import gym

from smarts.core.agent import Agent
from smarts.core.sensors import Observation

class ChaseViaPointsAgent(Agent):
    def act(self, obs: Observation):
        if (
            len(obs.via_data.near_via_points) < 1
            or obs.ego_vehicle_state.road_id != obs.via_data.near_via_points[0].road_id
        ):
            return (obs.waypoint_paths[0][0].speed_limit, 0)

        nearest = obs.via_data.near_via_points[0]
        if nearest.lane_index == obs.ego_vehicle_state.lane_index:
            return (nearest.required_speed, 0)

        return (
            nearest.required_speed,
            1 if nearest.lane_index > obs.ego_vehicle_state.lane_index else -1,
        )

In [ ]:
from smarts.core.agent import AgentSpec
from smarts.core.agent_interface import AgentInterface, AgentType
from smarts.core.utils.episodes import episodes
from smarts.env.wrappers.single_agent import SingleAgent

AGENT_ID = "SingleAgent"

def main(scenarios, sim_name, headless, num_episodes, seed, max_episode_steps=None):
    agent_spec = AgentSpec(
        interface=AgentInterface.from_type(
            AgentType.LanerWithSpeed, max_episode_steps=max_episode_steps
        ),
        agent_builder=ChaseViaPointsAgent,
    )

    env = gym.make(
        "smarts.env:hiway-v0",
        scenarios=scenarios,
        agent_specs={AGENT_ID: agent_spec},
        sim_name=sim_name,
        headless=headless,
        visdom=False,
        fixed_timestep_sec=0.1,
        sumo_headless=True,
        seed=seed,
    )

    # Wrap a single-agent env with SingleAgent wrapper to make `step` and `reset`
    # output compliant with gym spaces.
    env = SingleAgent(env)

    for episode in episodes(n=num_episodes):
        agent = agent_spec.build_agent()
        observation = env.reset()
        episode.record_scenario(env.scenario_log)

        done = False
        while not done:
            agent_action = agent.act(observation)
            observation, reward, done, info = env.step(agent_action)
            episode.record_step(observation, reward, done, info)

    env.close()

Run the example

In [ ]:
!scl envision start -s /content/SMARTS/scenarios

%%html
<iframe src="https://localhost:8081" width="100" height="100"></iframe>

In [ ]:
main(
    scenarios="/content/SMARTS/scenarios/loop",
    sim_name="Single agent",
    headless=True,
    num_episodes=10,
    seed=42,
)